In [ ]:
!pip install pulp
!pip install matplotlib

In [ ]:
import numpy as np
import random
from pulp import *


In [ ]:
#Function for normal distribution truncation:
from scipy.stats import truncnorm

def get_truncated_normal(mean, sd, low, upp):
    return truncnorm((low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)

In [ ]:
#Function to get the one-hot-encoded vectors for departure and arrival airports:

def one_hot_encode_airport(airport, num_airports):
    encoding = np.zeros(num_airports)
    encoding[airport] = 1
    return encoding

In [ ]:
#Generate full info for the arrival sides:

def generate_info_arv(requests):
    ts_arv = np.empty(shape=(len(requests),), dtype='object')
    for i in range(len(requests)):
        ts_arv[i] = requests[i][1] + requests[i][4]/5
        if ts_arv[i] > 287:
            ts_arv[i] = ts_arv[i] - 287
    return ts_arv

In [ ]:
#Modify the distribution based on historical data later:
def generate_scenario(number_of_requests, num_airports):

    #number_of_requests = 15000
    ts_72 = get_truncated_normal(mean=72, sd=60, low=0, upp=287).rvs(int(round(number_of_requests/2)))
    ts_72 = np.round(ts_72)

    ts_216 = get_truncated_normal(mean=216, sd=60, low=0, upp=287).rvs(int(round(number_of_requests/2)))
    ts_216 = np.round(ts_216)

    ts_dep = np.concatenate((ts_72, ts_216))
    ts_dep = ts_dep.astype(int)

    # ts_3 = get_truncated_normal(mean=3, sd=2, low=0, upp=287).rvs(int(round(number_of_requests*0.1)))
    # ts_3 = np.round(ts_3)

    # ts_130 = get_truncated_normal(mean=130, sd=24, low=0, upp=287).rvs(int(round(number_of_requests*0.7)))
    # ts_130 = np.round(ts_130)

    # ts_275 = get_truncated_normal(mean=275, sd=12, low=0, upp=287).rvs(int(round(number_of_requests*0.2)))
    # ts_275 = np.round(ts_275)

    # ts_3 = get_truncated_normal(mean=3, sd=2, low=0, upp=287).rvs(int(round(number_of_requests*0.05)))
    # ts_3 = np.round(ts_3)

    # ts_130 = get_truncated_normal(mean=130, sd=24, low=0, upp=287).rvs(int(round(number_of_requests*0.75)))
    # ts_130 = np.round(ts_130)

    # ts_275 = get_truncated_normal(mean=275, sd=12, low=0, upp=287).rvs(int(round(number_of_requests*0.2)))
    # ts_275 = np.round(ts_275)

    # ts_dep = np.concatenate((ts_3, ts_130, ts_275))
    # ts_dep = ts_dep.astype(int)

    #Generate index for requests:

    index = np.array(list(range(number_of_requests)))

    #Generate origin (0 and 1 are two considered origin airports, 2 represent other airports, encoded in one-hot vector):

    #num_airports = 3
    origin_airport = np.empty(shape=(number_of_requests,), dtype='object')
    destination_airport = np.empty(shape=(number_of_requests,), dtype='object')
    for i in range(number_of_requests):
        _org_airport = one_hot_encode_airport(random.randint(0,1), num_airports)
        _org_airport_list = _org_airport.tolist()
        origin_airport[i] = _org_airport_list
        #Generate destination (the destination will be different with the origin):
        _dest_airport = _org_airport.copy()
        while np.array_equal(_dest_airport, _org_airport):
            np.random.shuffle(_dest_airport)
        _dest_airport_list = _dest_airport.tolist()
        destination_airport[i] = _dest_airport_list

    #Generate flying time (assume between airport 0 and 1 is 2 hour, 0 to 2 and 1 to 2 is arbitrary):

    fly_time = np.empty(shape=(number_of_requests,), dtype='object')
    for i in range (number_of_requests):
        if origin_airport[i] == list([1.0, 0.0]) and destination_airport[i] == list([0.0, 1.0]):
            fly_time[i] = 120
        elif origin_airport[i] == list([0.0, 1.0]) and destination_airport[i] == list([1.0, 0.0]):
            fly_time[i] = random.choice([60, 120, 180])

    #Generate status cap:

    status_cap_dep = np.full((number_of_requests,), 0)
    status_cap_arv = np.full((number_of_requests,), 0)


    requests = np.stack((index, ts_dep, origin_airport, destination_airport, fly_time, status_cap_dep), axis=1)

    #Generate full info for the arv side:

    ts_arv = generate_info_arv(requests)

    #pseudo_belong_dep = np.full((number_of_requests,), 0)
    #pseudo_belong_arv = np.full((number_of_requests,), 0)

    # Define requests_full as dtype object
    num_entries = len(index)  # Given that 'index' is defined using np.array(list(range(number_of_requests)))
    # Create an empty array of the desired shape with dtype=object
    requests_full = np.empty((num_entries, 8), dtype=object)
    # Fill the array
    data = [index, ts_dep, origin_airport, destination_airport, fly_time, status_cap_dep, ts_arv, status_cap_arv]
    for i, column_data in enumerate(data):
        requests_full[:, i] = column_data

    return requests_full

In [ ]:
def generate_scenario_MILP(requests_full): #0 - departure slot, 1 - arrival slot, 2 - departure airport, 3 - arrival airport

  updated_requests = []

  for req in requests_full:
    updated_req = []
    updated_req.append(req[1])
    updated_req.append(req[6])
    updated_req.append(req[2].index(1))
    updated_req.append(req[3].index(1))
    updated_requests.append(updated_req)

  return updated_requests

In [ ]:
num_airports = 2
number_of_requests = 1000
requests = generate_scenario(number_of_requests, num_airports)
flight_requests = generate_scenario_MILP(requests)
time_slots = 288 # Time slots and their characteristics
capacity_per_slot = 6
max_movements = 6

# Create a MILP problem
problem = LpProblem(name="Flight_Scheduling", sense=LpMinimize)

In [ ]:
for i in range(10):
  print(flight_requests[i])

[4, 40.0, 1, 0]
[5, 29.0, 0, 1]
[3, 39.0, 1, 0]
[1, 25.0, 1, 0]
[4, 28.0, 0, 1]
[4, 28.0, 1, 0]
[2, 26.0, 0, 1]
[5, 29.0, 1, 0]
[2, 26.0, 1, 0]
[5, 41.0, 1, 0]


In [ ]:
# Creating binary decision variables for the departure slots of all flight requests
x = {(req, slot): LpVariable(
        name=f"x_{req}_{slot}", cat="Binary")
     for req in range(number_of_requests)
     for slot in range(time_slots)}

In [ ]:
# Creating binary decision variables for the arrival slots of all flight requests
y = {(req, slot): LpVariable(
        name=f"y_{req}_{slot}", cat="Binary")
     for req in range(number_of_requests)
     for slot in range(time_slots)}

In [ ]:
# Objective function - minimises the total absolute difference between the requested and allocated time interval
problem += lpSum((x[req, slot] * abs(slot - flight_requests[req][0])) for req in range(number_of_requests) for slot in range(time_slots))

In [ ]:
# Constraints -
# ensure that only one slot is assigned to a flight
for req in range(number_of_requests):
  problem += lpSum(x[req, slot] for slot in range(time_slots)) == 1
  problem += lpSum(y[req, slot] for slot in range(time_slots)) == 1

In [ ]:
# ensure slot interval between arrival and departure flight is equal to flying time
for req in range(number_of_requests):
  problem += lpSum((y[req, slot] * slot) for slot in range(time_slots)) - lpSum((x[req, slot] * slot) for slot in range(time_slots)) == flight_requests[req][1] - flight_requests[req][0]

In [ ]:
x_req_belong_airport_0 = []
for req in range(number_of_requests):
  if flight_requests[req][2] == 0:
    x_req_belong_airport_0.append(1)
  else:
    x_req_belong_airport_0.append(0)

y_req_belong_airport_0 = []
for req in range(number_of_requests):
  if flight_requests[req][3] == 0:
    y_req_belong_airport_0.append(1)
  else:
    y_req_belong_airport_0.append(0)

# airport capacity constraints, which limit the number of arrivals and departures at airports
for slot in range(time_slots):
    problem += lpSum((x[req,slot] * x_req_belong_airport_0[req] + y[req,slot] * y_req_belong_airport_0[req]) for req in range(number_of_requests)) <= capacity_per_slot

x_req_belong_airport_1 = []
for req in range(number_of_requests):
  if flight_requests[req][2] == 1:
    x_req_belong_airport_1.append(1)
  else:
    x_req_belong_airport_1.append(0)

y_req_belong_airport_1 = []
for req in range(number_of_requests):
  if flight_requests[req][3] == 1:
    y_req_belong_airport_1.append(1)
  else:
    y_req_belong_airport_1.append(0)

# airport capacity constraints, which limit the number of arrivals and departures at airports
for slot in range(time_slots):
    problem += lpSum((x[req,slot] * x_req_belong_airport_1[req] + y[req,slot] * y_req_belong_airport_1[req]) for req in range(number_of_requests)) <= capacity_per_slot


In [ ]:
import time

start_time = time.time()

# Solve the MILP problem
problem.solve(PULP_CBC_CMD(msg=True))

# Calculate elapsed time
elapsed_time = time.time() - start_time
print(f"Time taken to solve: {elapsed_time} seconds")

updated_slots = []
max_change = float('-inf')
requests_unchanged = 0

# Iterate through the decision variables and check if they are equal to 1
for req in range(number_of_requests):
  dep_slot = None
  arv_slot = None
  for slot in range(time_slots):
    if not dep_slot and x[req, slot].varValue == 1:
      dep_slot = slot
    if not arv_slot and y[req, slot].varValue == 1:
      arv_slot = slot
    if dep_slot and arv_slot:
      break
  change = abs(flight_requests[req][0] - dep_slot)
  if change > max_change:
    max_change = change
  if not change:
    requests_unchanged += 1
  updated_slots.append((req, dep_slot, arv_slot, change))

# Print the slot changes
for entry in updated_slots:
    print(f"Flight Request {entry[0]}: Departure slot changed from slot {flight_requests[entry[0]][0]} to slot {entry[1]}")

print("Status:", LpStatus[problem.status])
print(f"{requests_unchanged} flight requests out of {number_of_requests} were not shifted.")
print("Maximum Shift:", max_change)
print("Objective Value:", problem.objective.value())

In [ ]:
import matplotlib.pyplot as plt

original_slots = [req[0] for req in flight_requests if req[2] == 0]
original_slots.extend(req[1] for req in flight_requests if req[3] == 0)
print(len(original_slots))
# Count occurrences of each slot number
slot_counts = np.bincount(original_slots)

# Create x-axis values (timeslots)
timeslots = np.arange(len(slot_counts))

# Plotting the graph
plt.figure(figsize=(8, 6))
plt.bar(timeslots, slot_counts, align='center', alpha=0.7)
plt.xticks(timeslots)
plt.xlabel('Timeslot')
plt.ylabel('Count')
plt.title('Count of each timeslot at Airport 0 before')
plt.grid(True)
plt.show()

In [ ]:
new_slots = [entry[1] for entry in updated_slots if flight_requests[entry[0]][2] == 0]
new_slots.extend(entry[2] for entry in updated_slots if flight_requests[entry[0]][3] == 0)

# Count occurrences of each slot number
slot_counts = np.bincount(new_slots)

# Create x-axis values (timeslots)
timeslots = np.arange(len(slot_counts))

# Plotting the graph
plt.figure(figsize=(8, 6))
plt.bar(timeslots, slot_counts, align='center', alpha=0.7)
plt.xticks(timeslots)
plt.xlabel('Timeslot')
plt.ylabel('Count')
plt.title('Count of each timeslot at Airport 0 after')
plt.grid(True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

original_slots = [req[0] for req in flight_requests if req[2] == 1]
original_slots.extend(req[1] for req in flight_requests if req[3] == 1)
print(len(original_slots))
# Count occurrences of each slot number
slot_counts = np.bincount(original_slots)

# Create x-axis values (timeslots)
timeslots = np.arange(len(slot_counts))

# Plotting the graph
plt.figure(figsize=(8, 6))
plt.bar(timeslots, slot_counts, align='center', alpha=0.7)
plt.xticks(timeslots)
plt.xlabel('Timeslot')
plt.ylabel('Count')
plt.title('Count of each timeslot at Airport 1 before')
plt.grid(True)
plt.show()

In [ ]:
new_slots = [entry[1] for entry in updated_slots if flight_requests[entry[0]][2] == 1]
new_slots.extend(entry[2] for entry in updated_slots if flight_requests[entry[0]][3] == 1)

# Count occurrences of each slot number
slot_counts = np.bincount(new_slots)

# Create x-axis values (timeslots)
timeslots = np.arange(len(slot_counts))

# Plotting the graph
plt.figure(figsize=(8, 6))
plt.bar(timeslots, slot_counts, align='center', alpha=0.7)
plt.xticks(timeslots)
plt.xlabel('Timeslot')
plt.ylabel('Count')
plt.title('Count of each timeslot at Airport 1 after')
plt.grid(True)
plt.show()